In [257]:
import pandas as pd
import numpy as np

In [258]:
train=pd.read_csv('train.csv',lineterminator='\n')
test=pd.read_csv('20190520_test.csv',lineterminator='\n')

In [259]:
train.head(5)

,ID,review,label
0,1,Jo bhi ap se tou behtar hoon,Negative
1,2,ya Allah meri sister Affia ki madad farma,Positive
2,3,Yeh khud chahta a is umar main shadi krna. ha...,Negative
3,4,Tc ? Apky mun xe exe alfax achy nae lgty 😒💃,Negative
4,5,Good,Positive


In [260]:
X_train=train['review']
y_train=train['label']
X_test=test['review']
y_train=y_train.map({'Positive':1,'Negative':0})#将y_train映射为0，1

In [261]:
print(len(X_train))
print(y_train.shape)

6328
(6328,)


In [262]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')

In [263]:
def review_to_words(review):
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text() 
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    # 4. In Python, searching a set is much faster than searching a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    # 6. Join the words back into one string separated by space,and return the result.
    return( " ".join( meaningful_words ))   

In [264]:
train_X = [] 
test_X = []
for i in X_train:
    train_X.append(review_to_words(i))
for j in X_test:
    test_X.append(review_to_words(j))

In [265]:
print(train_X[1:3])
print(test_X[1:3])

['ya allah meri sister affia ki madad farma', 'yeh khud chahta umar main shadi krna ogi']
['jail road per firing se shakhs janbahaq', 'mehfil loot li aunty ne']


In [278]:
tfidf_vec=TfidfVectorizer()
X_tfidf_train=tfidf_vec.fit_transform(train_X)
X_tfidf_test=tfidf_vec.transform(test_X)

In [279]:
print(X_tfidf_train.shape)
print(tfidf_vec.get_feature_names()[1:100])

(6328, 17249)
['aaaa', 'aaainda', 'aaalam', 'aaalu', 'aaap', 'aaatma', 'aaaya', 'aabaadi', 'aabaai', 'aabad', 'aachi', 'aachuki', 'aadat', 'aadhi', 'aadmi', 'aadmiyon', 'aadmy', 'aae', 'aaeen', 'aaega', 'aaey', 'aaeyn', 'aafaq', 'aafia', 'aafiyat', 'aafrini', 'aag', 'aaga', 'aagah', 'aagahi', 'aagaye', 'aage', 'aaghaz', 'aagly', 'aagy', 'aagye', 'aahata', 'aahkoon', 'aai', 'aaid', 'aain', 'aainda', 'aaindah', 'aaini', 'aaiyd', 'aaiye', 'aaiyn', 'aaiyna', 'aaiynda', 'aaiyne', 'aaj', 'aaja', 'aajai', 'aajata', 'aajati', 'aajaye', 'aaka', 'aakar', 'aakhir', 'aakhirat', 'aakhirkar', 'aakhrat', 'aakhri', 'aakrr', 'aal', 'aala', 'aalaa', 'aalam', 'aalameen', 'aalikum', 'aalim', 'aalime', 'aalla', 'aalm', 'aalmi', 'aam', 'aama', 'aamaal', 'aamad', 'aamada', 'aamadah', 'aamal', 'aamdani', 'aameeeenn', 'aameen', 'aamen', 'aamesizh', 'aamezish', 'aamin', 'aamir', 'aamiriyat', 'aamiyana', 'aamla', 'aamm', 'aamriyat', 'aamriyaton', 'aamun', 'aan', 'aana']


In [280]:
mnb_tfidf=MultinomialNB()
mnb_tfidf.fit(X_tfidf_train,y_train)
mnb_tfidf_train_predict=mnb_tfidf.predict(X_tfidf_train)
print(mnb_tfidf.score(X_tfidf_train,y_train))
#这是预测结果
y_test=mnb_tfidf.predict(X_tfidf_test)
y_pred=mnb_tfidf.predict_proba(X_tfidf_test)

0.9208280657395702


In [281]:
#这是训练集的AUC结果
from sklearn.metrics import roc_curve,roc_auc_score
AUC=roc_auc_score(y_train,mnb_tfidf_train_predict)
print(AUC)

0.9179616563714295


In [282]:
print(y_pred)

[[0.58411642 0.41588358]
 [0.96297762 0.03702238]
 [0.4747661  0.5252339 ]
 ...
 [0.23155369 0.76844631]
 [0.30766134 0.69233866]
 [0.19215722 0.80784278]]


In [285]:
#这是预测该条文本的情感状态是否为「Positive」
y_predict_positive = [item[1] for item in y_pred]

In [286]:
#导出结果
ID = test['ID']
df={'ID':ID, 'Pred':y_predict_positive}
df=pd.DataFrame(df,columns=['ID', 'Pred'])
df.to_csv('result.csv',header=True,index=False) #写入文件